In [1]:
%run "Setup_Env.ipynb"

## Linking Multiple Chains Sequentially in LCEL

Here we will see how we can link several LLM Chains sequentially using LCEL.

Typically the output from one chain might go as input into the next chain and so on.

The overall chain would run each chain sequentially in order till we get the final output which can be a combination of intermediate outputs and inputs from the previous chains.

In [3]:
it_support_queue = [
    "I can't access my email. It keeps showing an error message. Please help.",
    "Tengo problemas con la VPN. No puedo conectarme a la red de la empresa. ¿Pueden ayudarme, por favor?",
    "Mon imprimante ne répond pas et n'imprime plus. J'ai besoin d'aide pour la réparer.",
    "我无法访问公司的网站。每次都显示错误信息。请帮忙解决。"
]

it_support_queue[0]

"I can't access my email. It keeps showing an error message. Please help."

In [4]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser

# Chain 1: Detect customer message language
prompt1 = """
  Act as a customer support agent.
  For the customer support message delimited below by triple backticks,
  Output the language of the message in one word only, e.g. Spanish

  Customer Message:
  ```{orig_msg}```
"""

prompt_template1 = ChatPromptTemplate.from_template(prompt1)

prompt1 = prompt_template1.invoke({"orig_msg": it_support_queue[0]})


In [7]:
detected_lang = chatgpt.invoke(prompt1)
detected_lang.content

'English'

In [8]:
llm_chain1 = (prompt_template1
                  |
              chatgpt
                  |
              StrOutputParser())

In [9]:
llm_chain1.invoke({'orig_msg': it_support_queue[1]})

'Spanish'

In [11]:
llm_chain1.invoke({'orig_msg': it_support_queue[2]})

'French'

In [14]:
from langchain_core.runnables import RunnablePassthrough, RunnableLambda, RunnableParallel

In [15]:
chain = RunnableParallel(
    orig_msg=RunnablePassthrough(),      # Keep original input
    orig_lang= llm_chain1  # Calculate length
)

# Test with a sample text - returns both original text and its length
chain.invoke(it_support_queue[1])

{'orig_msg': 'Tengo problemas con la VPN. No puedo conectarme a la red de la empresa. ¿Pueden ayudarme, por favor?',
 'orig_lang': 'Spanish'}

In [18]:
# The above can be also written as:

chain = {
        "orig_msg": RunnablePassthrough()
         } | RunnablePassthrough.assign(
    orig_lang=llm_chain1
)

chain.invoke(it_support_queue[1])

{'orig_msg': 'Tengo problemas con la VPN. No puedo conectarme a la red de la empresa. ¿Pueden ayudarme, por favor?',
 'orig_lang': 'Spanish'}

In [19]:
# Another way to do the same thing
RunnablePassthrough.assign(orig_lang=llm_chain1).invoke({'orig_msg': it_support_queue[1]})


{'orig_msg': 'Tengo problemas con la VPN. No puedo conectarme a la red de la empresa. ¿Pueden ayudarme, por favor?',
 'orig_lang': 'Spanish'}

In [20]:
# Chain 2: Translate Customer Message to English
prompt2 = """
  Act as a customer support agent.
  For the customer message and customer message language delimited below by triple backticks,
  Translate the customer message from the customer message language to English
  if customer message language is not in English,
  else return back the original customer message.

  Customer Message:
  ```{orig_msg}```
  Customer Message Language:
  ```{orig_lang}```
"""
prompt_template2 = ChatPromptTemplate.from_template(prompt2)
llm_chain2 = (prompt_template2
                  |
              chatgpt
                  |
              StrOutputParser())

In [21]:
# Chain 3: Generate a resolution response in English
prompt3 = """
  Act as a customer support agent.
  For the customer support message delimited below by triple backticks,
  Generate an appropriate resolution response in English.

  Customer Message:
  ```{trans_msg}```
"""
prompt_template3 = ChatPromptTemplate.from_template(prompt3)
llm_chain3 = (prompt_template3
                  |
              chatgpt
                  |
              StrOutputParser())

In [22]:
# Chain 4: Translate resolution response from English to Customer's original language
prompt4 = """
  Act as a customer support agent.
  For the customer resolution response and target language delimited below by triple backticks,
  Translate the customer resolution response message from English to the target language
  if target language is not in English,
  else return back the original customer resolution response.

  Customer Resolution Response:
  ```{trans_response}```
  Target Language:
  ```{orig_lang}```
"""
prompt_template4 = ChatPromptTemplate.from_template(prompt4)
llm_chain4 = (prompt_template4
                  |
              chatgpt
                  |
              StrOutputParser())

In [24]:
from langchain_core.runnables import RunnablePassthrough

final_chain = (
    RunnablePassthrough.assign(orig_lang=llm_chain1)
      |
    RunnablePassthrough.assign(trans_msg=llm_chain2)
      |
    RunnablePassthrough.assign(trans_response=llm_chain3)
      |
    RunnablePassthrough.assign(orig_response=llm_chain4)
)

In [26]:
response = final_chain.invoke({'orig_msg': it_support_queue[1]})
response

{'orig_msg': 'Tengo problemas con la VPN. No puedo conectarme a la red de la empresa. ¿Pueden ayudarme, por favor?',
 'orig_lang': 'Spanish',
 'trans_msg': "I have problems with the VPN. I can't connect to the company's network. Can you help me, please?",
 'trans_response': "Subject: Assistance with VPN Connection Issues\n\nDear [Customer's Name],\n\nThank you for reaching out to us regarding your VPN connection issues. I understand how important it is to access the company's network, and I'm here to help you resolve this matter.\n\nHere are a few troubleshooting steps you can try:\n\n1. **Check Your Internet Connection**: Ensure that you have a stable internet connection. You can try accessing other websites to confirm.\n\n2. **Restart the VPN Client**: Close the VPN application completely and then reopen it. Sometimes, a simple restart can resolve connection issues.\n\n3. **Verify Your Credentials**: Double-check that you are entering the correct username and password. If you have re

In [27]:
it_support_queue_formatted = [{'orig_msg': msg} for msg in it_support_queue]
responses = final_chain.map().invoke(it_support_queue_formatted)

In [28]:
import pandas as pd
pd.DataFrame(responses)

,orig_msg,orig_lang,trans_msg,trans_response,orig_response
0,I can't access my email. It keeps showing an e...,English,I can't access my email. It keeps showing an e...,Subject: Assistance with Email Access\n\nDear ...,Subject: Assistance with Email Access\n\nDear ...
1,Tengo problemas con la VPN. No puedo conectarm...,Spanish,I am having problems with the VPN. I cannot co...,Subject: Assistance with VPN Connection Issues...,```Subject: Asistencia con problemas de conexi...
2,Mon imprimante ne répond pas et n'imprime plus...,French,My printer is not responding and is no longer ...,"Hello,\n\nThank you for reaching out to us. I’...","```French\nBonjour,\n\nMerci de nous avoir con..."
3,我无法访问公司的网站。每次都显示错误信息。请帮忙解决。,Chinese,I cannot access the company's website. It alwa...,Subject: Assistance with Website Access\n\nDea...,```Chinese\n主题：网站访问帮助\n\n亲爱的[客户的名字]，\n\n感谢您联系我...
